***Parte B***: Redes Neuronales Convolucionales

In [ ]:
pip install torch torchvision matplotlib

***Preguntas***
1. ¿Que significa que el batch_size sea 4? Explica cómo afectaría al cálculo de los pesos en cada epoch.
La variable batch_size define cuantos datos se procesan en cada lote, al tener un valor pequeño el calculo de los pesos se realiza más veces en cada epoch.
2. ¿Por qué al dividir el conjunto de entrenamiento el parámetro shuffle está a True, mientras que el conjunto de test está a False?
El parametro shuffle reorganiza de manera aleatoria los datos que se cargan en cada epoch, esto puede ser beneficioso para evitar patrones que puedan afectar al desempeño del modelo, en el test no es necesario ya que son los datos de validación y por lo tanto no es necesario hacer el shuffle.
4. Indica la línea de código donde se realiza el cálculo de los gradientes para determinar cómo
modificar los pesos (los filtros kernel), e indica la línea de código donde se recalculan los pesos.
¿Qué se ha tenido que realizar antes para poder hacer el cálculo de gradientes y la
actualización de pesos?


In [ ]:
'''
Pregunta 3
'''
class Net(nn.Module):  # Define una clase para la red neuronal que hereda de nn.Module
    def __init__(self):  # Constructor de la clase
        super().__init__()  # Llama al constructor de la clase base nn.Module
        # Primera capa convolucional: toma imágenes con 3 canales (RGB), aplica 6 filtros de tamaño 5x5
        self.conv1 = nn.Conv2d(3, 6, 5)  
        # Capa de pooling: aplica max pooling con un tamaño de ventana 2x2 y un paso (stride) de 2
        self.pool = nn.MaxPool2d(2, 2)  
        # Segunda capa convolucional: toma 6 canales de entrada y aplica 16 filtros de tamaño 5x5
        self.conv2 = nn.Conv2d(6, 16, 5)  
        # Primera capa completamente conectada: conecta 16 * 5 * 5 entradas (salida de la convolución) a 120 neuronas
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  
        # Segunda capa completamente conectada: conecta 120 neuronas a 84 neuronas
        self.fc2 = nn.Linear(120, 84)  
        # Tercera capa completamente conectada: conecta 84 neuronas a 10 neuronas (una por cada clase)
        self.fc3 = nn.Linear(84, 10)  

    def forward(self, x):  # Define el flujo de datos a través de la red (forward propagation)
        # Aplica la primera capa convolucional, seguida de ReLU y max pooling
        x = self.pool(F.relu(self.conv1(x)))  
        # Aplica la segunda capa convolucional, seguida de ReLU y max pooling
        x = self.pool(F.relu(self.conv2(x)))  
        # Aplana el tensor (convierte las dimensiones de características en una sola dimensión, excepto el batch)
        x = torch.flatten(x, 1)  # Mantiene la dimensión del batch (dimensión 0)
        # Aplica la primera capa completamente conectada con activación ReLU
        x = F.relu(self.fc1(x))  
        # Aplica la segunda capa completamente conectada con activación ReLU
        x = F.relu(self.fc2(x))  
        # Aplica la última capa completamente conectada (sin activación, ya que es la salida final)
        x = self.fc3(x)  
        return x  # Devuelve la salida final (logits para las 10 clases)